# K-Nearest Neighbors (K-NN)

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import codecs
import re

## Importing the dataset
* 從三個資料夾中讀取資料

In [2]:
all_data=[]
paths =[r'spam_data/spam', r'spam_data/easy_ham', r'spam_data/hard_ham'] 
for path in paths:
    for fn in glob.glob(path+"/*"):
        if "ham" not in fn:
            is_spam = 1
        else:
            is_spam = 0
        #codecs.open可以避開錯誤，用errors='ignore'
        with codecs.open(fn,encoding='utf-8', errors='ignore') as file:
            for line in file:
                #這個line的開頭為Subject:
                if line.startswith("Subject:"):
                    subject=re.sub(r"^Subject:","",line).strip()
                    all_data.append([subject,is_spam])
all_data = np.array(all_data)

### 取出訓練內文與標註

In [3]:
X = all_data[:,0]
Y = all_data[:,1].astype(np.uint8)

In [4]:
print('Training Data Examples : \n{}'.format(X[:5]))

Training Data Examples : 
['Life Insurance - Why Pay More?'
 '[ILUG] Guaranteed to lose 10-12 lbs in 30 days 10.206'
 'Guaranteed to lose 10-12 lbs in 30 days                          11.150'
 'Re: Fw: User Name & Password to Membership To 5 Sites zzzz@example.com pviqg'
 '[ILUG-Social] re: Guaranteed to lose 10-12 lbs in 30 days 10.148']


In [5]:
print('Labeling Data Examples : \n{}'.format(Y[:5]))

Labeling Data Examples : 
[1 1 1 1 1]


### 文字預處理
* 細節可以參考前面章節

In [6]:
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords

import nltk

nltk.download('stopwords')

# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

## 創建Lemmatizer
lemmatizer = WordNetLemmatizer() 
def get_wordnet_pos(word):
    """將pos_tag結果mapping到lemmatizer中pos的格式"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def clean_content(X):
    # remove non-alphabet characters
    X_clean = [re.sub('[^a-zA-Z]',' ', x).lower() for x in X]
    # tokenize
    X_word_tokenize = [nltk.word_tokenize(x) for x in X_clean]
    # stopwords_lemmatizer
    X_stopwords_lemmatizer = []
    stop_words = set(stopwords.words('english'))
    for content in X_word_tokenize:
        content_clean = []
        for word in content:
            if word not in stop_words:
                word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
                content_clean.append(word)
        X_stopwords_lemmatizer.append(content_clean)
    
    X_output = [' '.join(x) for x in X_stopwords_lemmatizer]
    
    return X_output
                 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeff.yang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
X = clean_content(X)

### Bag of words

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
#max_features是要建造幾個column，會按造字出現的頻率高低去篩選，1500並沒有特別含義，大家可以自己嘗試不同數值或不加入限制
cv=CountVectorizer(max_features = 1500)
X=cv.fit_transform(X).toarray()

In [9]:
# 有 3423個樣本，每個樣本用1500維表示
X.shape 

(3423, 1500)

## 將資料拆成 train/test set

In [10]:
from sklearn.model_selection import train_test_split
# random_state 是為了讓各為學員得到相同的結果，平時可以移除
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

## Training the K-NN model on the Training set

In [18]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

## 測試 train/testset的 Accuracy

In [19]:
print('Trainset Accuracy: {}'.format(classifier.score(X_train, y_train)))

Trainset Accuracy: 0.91672753834916


In [20]:
print('Testset Accuracy: {}'.format(classifier.score(X_test, y_test)))

Testset Accuracy: 0.8875912408759125


## 獲得 testset 上的結果

In [22]:
y_pred = classifier.predict(X_test)

## Making the Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[581   6]
 [ 71  27]]


0.8875912408759125

### 運用K-fold尋找適合K值

In [13]:
# Applying k-Fold Cross Validation
#n-jobs=-1，是指cpu全開
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
n_neighbors  = [5, 10, 25, 50, 100] ## 可自行嘗試不同K值
for k in n_neighbors:
    classifier = KNeighborsClassifier(n_neighbors = k, metric = 'minkowski', p = 2)
    # cv = 10 代表切成10等分
    accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,n_jobs=-1)
    
    print('設置K值:{}'.format(k))
    print('Average Accuracy: {}'.format(accuracies.mean()))
    print('Accuracy STD: {}'.format(accuracies.std()))

設置K值:5
Average Accuracy: 0.880947378655408
Accuracy STD: 0.011561254412983857
設置K值:10
Average Accuracy: 0.8623193886405567
Accuracy STD: 0.006510165116834898
設置K值:25
Average Accuracy: 0.8528182474459847
Accuracy STD: 0.0024837372232837055
設置K值:50
Average Accuracy: 0.8520856467133839
Accuracy STD: 0.0014068907077920568
設置K值:100
Average Accuracy: 0.8520856467133839
Accuracy STD: 0.0014068907077920568
